In [1]:
import psycopg2
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
df = pd.read_csv('bnf.csv')
symbol = 'BANKNIFTY'

In [3]:
df['date_timestamp'] = pd.to_datetime(df['date_timestamp'], format='%d-%m-%Y %H:%M')
df['open'] = df['open'].astype(int)
df['low'] = df['low'].astype(int)
df['high'] = df['high'].astype(int)
df['close'] = df['close'].astype(int)
df['volume'] = df['volume'].astype(int)
df['expiry'] = pd.to_datetime(df['expiry'], format='%d-%m-%Y %H:%M')
df['expiry_type'] = df['expiry_type'].astype(str)
df['id'] = df['id'].astype(int)

# Display the DataFrame to verify changes
#print(df.head())

       date_timestamp     symbol     open      low     high    close  volume  \
0 2022-05-02 09:15:00  BANKNIFTY  3572495  3567375  3587240  3586505   96000   
1 2022-05-02 09:16:00  BANKNIFTY  3586600  3586525  3595000  3595000   52075   
2 2022-05-02 09:17:00  BANKNIFTY  3595705  3588730  3595900  3590030   39125   
3 2022-05-02 09:18:00  BANKNIFTY  3590265  3590265  3597350  3596000   36250   
4 2022-05-02 09:19:00  BANKNIFTY  3596000  3595960  3603220  3602500   38450   

               expiry expiry_type        id  
0 2022-05-26 14:30:00           I  23751303  
1 2022-05-26 14:30:00           I  23751304  
2 2022-05-26 14:30:00           I  23751305  
3 2022-05-26 14:30:00           I  23751306  
4 2022-05-26 14:30:00           I  23751307  


In [27]:
nft = df[df['expiry']=='29-06-2023 14:30']
nft

,date_timestamp,symbol,open,low,high,close,volume,expiry,expiry_type,id
105750,2023-05-26 09:15:00,BANKNIFTY,4376525,4374000,4382130,4374000,35875,2023-06-29 14:30:00,I,48739443
105751,2023-05-26 09:15:00,BANKNIFTY,4376525,4374000,4382130,4374000,35875,2023-06-29 14:30:00,I,1944540
105752,2023-05-26 09:16:00,BANKNIFTY,4374060,4371495,4375735,4373500,72725,2023-06-29 14:30:00,I,1944541
105753,2023-05-26 09:16:00,BANKNIFTY,4374060,4371495,4375735,4373500,72725,2023-06-29 14:30:00,I,48739444
105754,2023-05-26 09:17:00,BANKNIFTY,4372000,4371070,4374325,4371995,20175,2023-06-29 14:30:00,I,48739445
...,...,...,...,...,...,...,...,...,...,...
122995,2023-06-27 15:27:00,BANKNIFTY,4411700,4411500,4412000,4411885,14875,2023-06-29 14:30:00,I,50320713
122996,2023-06-27 15:28:00,BANKNIFTY,4412000,4411600,4412330,4411800,16800,2023-06-29 14:30:00,I,50320714
122997,2023-06-27 15:28:00,BANKNIFTY,4412000,4411600,4412330,4411800,16800,2023-06-29 14:30:00,I,3525811
122998,2023-06-27 15:29:00,BANKNIFTY,4411700,4411500,4413255,4412995,15625,2023-06-29 14:30:00,I,3525812


In [ ]:

short_window = 9
long_window = 26
nft['Short_EMA'] = nft['close'].ewm(span=short_window, adjust=False).mean()
nft['Long_EMA'] = nft['close'].ewm(span=long_window, adjust=False).mean()

nft['Signal'] = 0
nft['Position'] = 0


In [30]:
for i in range(1, len(nft)):
    if nft['Short_EMA'].iloc[i] > nft['Long_EMA'].iloc[i] and nft['Short_EMA'].iloc[i-1] <= nft['Long_EMA'].iloc[i-1]:
        nft.at[nft.index[i], 'Signal'] = 1  # Buy signal
    elif nft['Short_EMA'].iloc[i] < nft['Long_EMA'].iloc[i] and nft['Short_EMA'].iloc[i-1] >= nft['Long_EMA'].iloc[i-1]:
        nft.at[nft.index[i], 'Signal'] = -1  # Sell signal

nft.head()


,date_timestamp,symbol,open,low,high,close,volume,expiry,expiry_type,id,Short_EMA,Long_EMA,Signal,Position
105750,2023-05-26 09:15:00,BANKNIFTY,4376525,4374000,4382130,4374000,35875,2023-06-29 14:30:00,I,48739443,4374000.0,4.374000e+06,0,0
105751,2023-05-26 09:15:00,BANKNIFTY,4376525,4374000,4382130,4374000,35875,2023-06-29 14:30:00,I,1944540,4374000.0,4.374000e+06,0,0
105752,2023-05-26 09:16:00,BANKNIFTY,4374060,4371495,4375735,4373500,72725,2023-06-29 14:30:00,I,1944541,4373900.0,4.373963e+06,-1,0
105753,2023-05-26 09:16:00,BANKNIFTY,4374060,4371495,4375735,4373500,72725,2023-06-29 14:30:00,I,48739444,4373820.0,4.373929e+06,0,0
105754,2023-05-26 09:17:00,BANKNIFTY,4372000,4371070,4374325,4371995,20175,2023-06-29 14:30:00,I,48739445,4373455.0,4.373785e+06,0,0


In [31]:

fig = make_subplots(rows=1, cols=1)

# Candlestick chart
candlestick = go.Candlestick(x=nft.index,
                             open=nft['open'],
                             high=nft['high'],
                             low=nft['low'],
                             close=nft['close'],
                             name='Candlesticks')
fig.add_trace(candlestick)

# Short EMA
short_ema = go.Scatter(x=nft.index, y=nft['Short_EMA'], mode='lines', name='Short EMA')
fig.add_trace(short_ema)

# Long EMA
long_ema = go.Scatter(x=nft.index, y=nft['Long_EMA'], mode='lines', name='Long EMA')
fig.add_trace(long_ema)

# Buy signals
buy_signals = go.Scatter(x=nft[nft['Signal'] == 1].index, 
                         y=nft['Short_EMA'][nft['Signal'] == 1], 
                         mode='markers', 
                         marker=dict(symbol='triangle-up', color='blue', size=5), 
                         name='Buy Signal')
fig.add_trace(buy_signals)

# Sell signals
sell_signals = go.Scatter(x=nft[nft['Signal'] == -1].index, 
                          y=nft['Short_EMA'][nft['Signal'] == -1], 
                          mode='markers', 
                          marker=dict(symbol='triangle-down', color='black', size=5), 
                          name='Sell Signal')
fig.add_trace(sell_signals)


fig.update_layout(title=f'{symbol} EMA Crossover Strategy',
                  yaxis_title='Price',
                  xaxis_title='Date',
                  xaxis_rangeslider_visible=False,
                  width=1440,  # Adjust the width as needed
                  height=400)  # Adjust the height as needed

# Show plot
fig.show()


In [32]:

position_size = 1  
current_position = 0  # Track current position (1 for long, -1 for short, 0 for neutral)
entry_price = 0  # Track entry price
total_pnl = 0  # Total P&L

# Iterate through each row in the DataFrame
for index, row in nft.iterrows():
    if row['Signal'] == 1 and current_position == 0:  # Buy signal and no current position
        current_position = 1
        # print("entry")
        entry_price = row['close']
    elif row['Signal'] == -1 and current_position == 1:  # Sell signal and long position
        pnl = (row['close'] - entry_price) * position_size
        total_pnl += pnl
        # ###print("close")
        current_position = 0
        entry_price = 0
    elif row['Signal'] == -1 and current_position == 0:  # Sell signal but no current long position
        continue  # Can't sell if not long
        
# Print total P&L
print(f'Total P&L: {total_pnl}')

Total P&L: -123820


In [35]:
dfo = pd.read_csv('bnf_opt.csv')

options = dfo[dfo['expiry']=='29-06-2023']

options

,date_timestamp,symbol,open,high,low,close,volume,opt_type,strike,expiry_type,id,expiry
576978,26-05-2023 09:15,BANKNIFTY,1610,2050,1610,1980,225,CE,4950000,I,15175645,29-06-2023
576979,26-05-2023 09:15,BANKNIFTY,1900,1900,1900,1900,75,PE,3800000,I,15525108,29-06-2023
576980,26-05-2023 09:15,BANKNIFTY,10045,10045,9140,9495,300,PE,4110000,I,15527411,29-06-2023
576981,26-05-2023 09:15,BANKNIFTY,13760,14940,13315,14000,1275,PE,4160000,I,15528469,29-06-2023
576982,26-05-2023 09:15,BANKNIFTY,39885,40345,37500,39980,3075,PE,4300000,I,15532521,29-06-2023
...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,22-06-2023 10:50,BANKNIFTY,1205,1205,1175,1175,300,PE,4150000,I,31340074,29-06-2023
1048571,22-06-2023 10:50,BANKNIFTY,54135,54135,53790,53790,1300,PE,4440000,I,31355891,29-06-2023
1048572,22-06-2023 10:50,BANKNIFTY,2380,2380,2320,2320,300,PE,4240000,I,31343139,29-06-2023
1048573,22-06-2023 10:50,BANKNIFTY,18050,18150,17900,17950,4500,PE,4370000,I,31350644,29-06-2023


In [40]:
copt = options[options['opt_type']=='CE']
popt = options[options['opt_type']=='PE']


In [41]:
df['date_timestamp'] = pd.to_datetime(df['date_timestamp'])
copt['date_timestamp'] = pd.to_datetime(copt['date_timestamp'])
popt['date_timestamp'] = pd.to_datetime(popt['date_timestamp'])

C:\Users\tanma\AppData\Local\Temp\ipykernel_6552\920306415.py:2: UserWarning:

Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.

C:\Users\tanma\AppData\Local\Temp\ipykernel_6552\920306415.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tanma\AppData\Local\Temp\ipykernel_6552\920306415.py:3: UserWarning:

Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.

C:\Users\tanma\AppData\Local\Temp\ipykernel_6552\920306415.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [47]:

# Initialize lists to keep track of trades and PnL
trades = []
current_positions = {'call': None, 'put': None}
pnl = 0

# Loop through each row in the futures DataFrame
for i, row in nft.iterrows():
    signal = row['Signal']
    date_timestamp = row['date_timestamp']
    future_close = row['close']
    
    # print(f"Processing {date_timestamp} with signal {signal} and future close {future_close}")

    if signal == 1:  # Buy signal
        # Find the ATM call option closest to the future's closing price
        atm_call = copt[(copt['date_timestamp'] == date_timestamp) & 
                        (abs(copt['strike'] - future_close) == abs(copt['strike'] - future_close).min())]

        # Find the ATM put options to short if they exist
        atm_put = popt[(popt['date_timestamp'] == date_timestamp) & 
                       (abs(popt['strike'] - future_close) == abs(popt['strike'] - future_close).min())]

        # print("ATM Call Options: ", atm_call)
        # print("ATM Put Options: ", atm_put)

        # Record the trades
        if not atm_call.empty:
            trades.append({
                'date': date_timestamp,
                'action': 'buy',
                'option_type': 'call',
                'strike': atm_call.iloc[0]['strike'],
                'price': atm_call.iloc[0]['close']
            })
            # Update current positions
            current_positions['call'] = atm_call.iloc[0]['close']
        if not atm_put.empty:
            trades.append({
                'date': date_timestamp,
                'action': 'short',
                'option_type': 'put',
                'strike': atm_put.iloc[0]['strike'],
                'price': atm_put.iloc[0]['close']
            })
            # Update current positions
            current_positions['put'] = atm_put.iloc[0]['close']
    
    elif signal == -1:  # Sell signal
        # Find the ATM put option closest to the future's closing price
        atm_put = popt[(popt['date_timestamp'] == date_timestamp) & 
                       (abs(popt['strike'] - future_close) == abs(popt['strike'] - future_close).min())]

        # Find the ATM call options to square off if they exist
        atm_call = copt[(copt['date_timestamp'] == date_timestamp) & 
                        (abs(copt['strike'] - future_close) == abs(copt['strike'] - future_close).min())]

        # print("ATM Put Options: ", atm_put)
        # print("ATM Call Options: ", atm_call)

        # Record the trades and calculate PnL
        if not atm_put.empty:
            trades.append({
                'date': date_timestamp,
                'action': 'buy',
                'option_type': 'put',
                'strike': atm_put.iloc[0]['strike'],
                'price': atm_put.iloc[0]['close']
            })
            # Calculate PnL if we previously shorted a put
            if current_positions['put'] is not None:
                pnl += current_positions['put'] - atm_put.iloc[0]['close']
                current_positions['put'] = None  # Close the position
        if not atm_call.empty:
            trades.append({
                'date': date_timestamp,
                'action': 'squareoff',
                'option_type': 'call',
                'strike': atm_call.iloc[0]['strike'],
                'price': atm_call.iloc[0]['close']
            })
            # Calculate PnL if we previously bought a call
            if current_positions['call'] is not None:
                pnl += atm_call.iloc[0]['close'] - current_positions['call']
                current_positions['call'] = None  # Close the position

# Convert trades list to DataFrame for analysis
trades_df = pd.DataFrame(trades)

# Display the trades and total PnL
print(trades_df)
print(f"Total PnL: {pnl}")

                    date     action option_type   strike  price
0    2023-05-26 09:16:00        buy         put  4370000  67765
1    2023-05-26 09:16:00  squareoff        call  4370000  70915
2    2023-05-26 09:29:00        buy        call  4370000  69905
3    2023-05-26 09:29:00      short         put  4370000  67075
4    2023-05-26 09:35:00        buy         put  4370000  68145
...                  ...        ...         ...      ...    ...
1344 2023-06-22 10:22:00      short         put  4400000  31075
1345 2023-06-22 10:40:00        buy         put  4400000  30750
1346 2023-06-22 10:40:00  squareoff        call  4400000  26865
1347 2023-06-22 10:41:00        buy        call  4400000  27725
1348 2023-06-22 10:41:00      short         put  4400000  29855

[1349 rows x 5 columns]
Total PnL: 167060


In [48]:

# Ensure trades_df is sorted by date
trades_df = trades_df.sort_values(by='date')

# Calculate daily returns
trades_df['daily_return'] = trades_df['price'].pct_change()

# Fill NaN values with 0 for the first trade
trades_df['daily_return'] = trades_df['daily_return'].fillna(0)

# Calculate cumulative returns
trades_df['cumulative_return'] = (1 + trades_df['daily_return']).cumprod() - 1

# Assume risk-free rate is 0 for simplicity
risk_free_rate = 0

# Calculate the mean and standard deviation of daily returns
mean_daily_return = trades_df['daily_return'].mean()
std_daily_return = trades_df['daily_return'].std()

# Calculate the Sharpe ratio
sharpe_ratio = (mean_daily_return - risk_free_rate) / std_daily_return

# Calculate the total PnL
total_pnl = pnl

# Assume initial investment is the sum of all buy trades
initial_investment = trades_df[trades_df['action'].isin(['buy', 'short'])]['price'].sum()

# Calculate percentage PnL
percentage_pnl = (total_pnl / initial_investment) * 100

# Display the results
print(f"Sharpe Ratio: {sharpe_ratio}")
print(f"Percentage PnL: {percentage_pnl}%")


Sharpe Ratio: 0.019993927283579172
Percentage PnL: 0.33622513921916514%
